# Generation: Generating a Response

In [1]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [2]:
pip show langchain

Name: langchain
Version: 1.2.3
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: C:\Premasis\Development\Pycharm\LangChainSample\.venv\Lib\site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext dotenv
%dotenv

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [5]:
vectorstore = Chroma(persist_directory = "./local-database",
                     embedding_function = OpenAIEmbeddings(model='text-embedding-ada-002'))

In [6]:
len(vectorstore.get()['documents'])

20

In [7]:
retriever = vectorstore.as_retriever(search_type = 'mmr', 
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

In [8]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [9]:
chat = ChatOpenAI(
    model="gpt-4",
    seed=365,
    max_tokens=250,
)


In [10]:
question = "What software do data scientists use?"

In [11]:
chain = ({'context': retriever, 
         'question': RunnablePassthrough()} 
         | prompt_template 
         | chat 
         | StrOutputParser())

In [12]:
chain.invoke(question)

'Data scientists use several software and programming languages. The two most popular tools are R and Python, which are lauded for their adaptability and their integration within multiple data and data science software platforms. These tools are not just suitable for mathematical and statistical computations but can solve a wide variety of business and data-related problems. Another crucial software framework used in data science is Hadoop, designed particularly to handle the complexity and computational intensity of big data. Moreover, Power BI, SaS, Qlik, and Tableau are top-notch examples of software designed for business intelligence visualizations.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need'